要正确理解误差反向传播法，我个人认为有两种方法：一种是基于数学式；另一种是基于计算图

## 计算图
用计算图解题的情况下，需要按如下流程进行。
1.构建计算图。
2.在计算图上，从左向右进行计算。

这里的第2歩“从左向右进行计算”是一种正方向上的传播，简称为**正向传播**（forward propagation）。
**反向传播**即以反方向传播

使用计算图最大的原因是，可以通过反向传播高效计算导数

计算图的反向传播：沿着与正方向相反的方向，乘上局部导数

### 链式法则
> 如果某个函数由复合函数表示，则该复合函数的导数可以用构成复合函数的各个函数的导数的乘积表示。

如果
$$
z = t^2\\
t = x + y
$$
那么：
$$
\frac{\partial z}{\partial x} = \frac{\partial z}{\partial t} \frac{\partial t}{\partial x} = 2t=2(x + y)
$$

---
反向传播是基于链式法则的:

![](./%E5%9B%BE5-7%20%E9%93%BE%E5%BC%8F%E6%B3%95%E5%88%99%E4%B8%8E%E8%AE%A1%E7%AE%97%E5%9B%BE.png)

![](./%E5%9B%BE5-8%20%E9%93%BE%E5%BC%8F%E6%B3%95%E5%88%99%E4%B8%8E%E8%AE%A1%E7%AE%97%E5%9B%BE.png)




## 反向传播
加法节点的反向传播将上游的值原封不动地输出到下游  
乘法的反向传播会乘以输入信号的翻转值，乘法的反向传播需要正向传播时的输入信号值。因此，实现乘法节点的反向传播时，要保存正向传播的输入信号


## 乘法层和加法层的实现

In [ ]:
# 乘法层 需要记住输入
class MulLayer:
  def __init__(self):
    self.x = None
    self.y = None

  def forward(self, x, y):
    self.x = x
    self.y = y
    out = x * y
    return out

  def backward(self, dout):
    dx = dout * self.y
    dy = dout * self.x
    return dx, dy

# 加法层 不需要记住输入
class AddLayer:
  def __init__(self):
    pass

  def forward(self, x, y):
    out = x + y
    return out

  def backward(self, dout):
    dx = dout * 1
    dy = dout * 1
    return dx, dy
